In [23]:
# Import modules
from selenium import webdriver
from selenium.webdriver.common.by import By
import smtplib
import time
import configparser
import pandas as pd
import uuid
from datetime import datetime
import json
import os

# Set maximum number of rows and columns to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Create webdriver object
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [9]:
# Read email username and password from configuration file
config = configparser.ConfigParser()
config.read('config.cfg')
sender_email = config.get('Email', 'username')
sender_password = config.get('Email', 'password')

In [18]:
# Define function to check price
def check_price(products):
    prices = pd.DataFrame(columns=['id', 'product', 'url', 'price', 'datetime'])
    for product in products:
        urls = products[product]["urls"]
        desired_price = products[product]["desired_price"]
        for url in urls:
            # Load product page
            driver.get(url)
            print(f"Checking price for {product} at {url}")
            # Find price element
            try:
                price_element = driver.find_element(By.CLASS_NAME, 'uniform-banner-box-price')
            except:
                price_element = driver.find_element(By.CLASS_NAME, 'product-price-value')
            # Get price text and convert to float
            price_text = price_element.text.replace("R$ ", "").replace('.', '').replace(',', '.')
            price = float(price_text)
            # Add price and datetime to DataFrame
            now = datetime.now()
            prices = pd.concat([prices, pd.DataFrame({'id': [str(uuid.uuid4())], 'product': [product], 'url': [url], 'price': [price], 'datetime': [now]})], ignore_index=True)
    
    # Write prices to parquet file
    file_path = '/mnt/c/Users/mahei/OneDrive/Documentos/Python/monitor-aliexpress-product-price/prices.parquet'
    if not os.path.isfile(file_path):
        prices.to_parquet(file_path, engine='fastparquet')
    else:
        prices.to_parquet(file_path, engine='fastparquet', append=True)
    
    # Return DataFrame of prices
    return prices

In [30]:
def check_previous_price(lowest_price, product):
    # Read previous prices from parquet file
    file_path = '/mnt/c/Users/mahei/OneDrive/Documentos/Python/monitor-aliexpress-product-price/prices.parquet'
    if os.path.isfile(file_path):
        previous_prices = pd.read_parquet(file_path, engine='fastparquet')
        previous_prices = previous_prices[previous_prices['product'] == product]
    else:
        return None
    # Find previous lowest price for product
    previous_lowest_price = previous_prices[previous_prices['price'] < lowest_price].sort_values('datetime', ascending=False).head(1)
    if previous_lowest_price.empty:
        return None
    else:
        url = previous_lowest_price['url'].iloc[0]
        price = previous_lowest_price['price'].iloc[0]
        return {'product': product, 'url': url, 'price': price}

In [14]:
def send_email_alert(prices, products):
    message = "Subject: Price Alert\n\n"
    for product, group in prices.groupby('product'):
        lowest_price = group['price'].min()
        previous_price = check_previous_price(lowest_price, product)
        for index, row in group.iterrows():
            price = row["price"]
            url = row["url"]
            if previous_price is None or price < previous_price['price']:
                message += f"The {product} you are tracking is now R$ {price}.\n{url}\n\n"
    if "Price Alert" not in message:
        return
    print("Sending email alert...")
    # Create SMTP object
    server = smtplib.SMTP("smtp.gmail.com", 587)
    # Start TLS encryption
    server.starttls()
    # Login to email account
    server.login(sender_email, sender_password)
    # Send email message
    server.sendmail(sender_email, sender_email, message)
    # Quit SMTP object
    server.quit()

In [16]:
with open('/mnt/c/Users/mahei/OneDrive/Documentos/Python/monitor-aliexpress-product-price/products.json', 'r') as f:
    products = json.load(f)

In [19]:
current_prices = check_price(products)

Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005003359522996.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005004667422776.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005004643544273.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005004767710064.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005003391597856.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005003391612933.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005003391623760.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005004586709793.html
Checking price for Ryzen 9 5950X at https://pt.aliexpress.com/item/1005004305762942.html


In [29]:
for product, group in current_prices.groupby('product'):
        lowest_price = group['price'].min()
        print(lowest_price,product)
        # previous_price = check_previous_price(lowest_price, product)

2640.06 Ryzen 9 5950X


In [37]:
lowest_price = 2900.00

In [12]:
file_path = '/mnt/c/Users/mahei/OneDrive/Documentos/Python/monitor-aliexpress-product-price/prices.parquet'
previous_prices = pd.read_parquet(file_path, engine='fastparquet')

df = previous_prices


In [8]:
# Sort the dataframe by 'datetime' column in ascending order
df = df.sort_values('datetime')

In [13]:
df['prev_price'] = df.groupby(['product', 'url'])['price'].shift(1)
df['price_diff'] = df['price'] - df['prev_price']
df = df[df["price_diff"] != 0]

In [ ]:
df

In [34]:
df.groupby(['product', df['datetime'].dt.date]).agg({'price':'min'})

price
product         datetime           
Ryzen 7 5800X3D 2023-03-02  1790.77
                2023-03-04  1927.42
                2023-03-06  1921.76
Ryzen 9 5950X   2023-03-02  2640.06
                2023-03-04  2650.11
                2023-03-06  2642.40